# Classifier
> This is the class that creates the dictionary model

In [1]:
#| default_exp Classifier

In [ ]:
#| hide
from nbdev.showdoc import *


In [50]:
#| export
import pandas as pd
import numpy as np

In [23]:
#negative_words file
df = pd.read_csv('negative_words.csv')
df = df.drop(columns=['original']) #drop it since it contains the concept with an * in it

neg_words = [] 
for _, row in df.iterrows():
    for item in row:
        if pd.notnull(item):
            neg_words.append(item)


In [32]:
#Drop duplicates in neg_words:
neg_words = list(set(neg_words))

len(neg_words)

1599

In [47]:
#moral_foundations_dictionary file:
df = pd.read_csv('moral_foundations_dictionary_1.0.csv')
df = df[['categories','word_examples']]

virtue = []
vice = []
for _, row in df.iterrows():
    if pd.notnull(row['word_examples']):
        words = row['word_examples'].split(',')
        if 'Virtue' in row['categories']:
            virtue.extend(words)
        elif 'Vice' in row['categories']:
            vice.extend(words)
        else:
            print(words)




['moralidad']
['moral']
['etica']
['etico']
['principios']
['valores']
['bueno', ' buena']
['bondadoso']
['correcto']
['erroneo', ' equivocado', ' incorrecto', ' malo']
['justicia']
['fechoria', ' pecado', ' indebido']
['virtud']
['vicioso']
['moralidad']
['etica']
vice examples:  ['destruccion', ' perjudicar', 'perjudicar', ' ataque', 'explota', ' abusivo', 'abandonar', ' matar', 'fanatismo', 'plantado']
virtue examples:  [' justicia', 'comunal', 'miembro', 'justicia', 'permitir', ' simpatizar', 'modestia', 'posicion', 'diferir', ' sometido']


In [48]:
virtue.extend(['moralidad','moral','etica','etico','principios','valores','bueno','buena','bondadoso','correcto','justicia','virtud','moralidad','etica'])
vice.extend(['erroneo','equivocado','incorrecto','malo','fechoria','pecado','indebido','vicioso'])
#Drop duplicates in virtue and vice:
virtue = list(set(virtue))
vice = list(set(vice))

print('vice examples: ', vice[:10])
print('virtue examples: ', virtue[:10])

vice examples:  ['incorrecto', 'destruccion', ' perjudicar', ' ataque', 'perjudicar', 'explota', ' abusivo', 'abandonar', ' matar', 'fanatismo']
virtue examples:  [' justicia', 'comunal', 'miembro', 'justicia', 'modestia', 'permitir', ' simpatizar', 'posicion', 'diferir', ' sometido']


In [56]:
#Load the vectors data frame:
def load_embeddings(file_path):
    word_to_vec = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            word_to_vec[word] = vector
    return word_to_vec

word_to_vec = load_embeddings('vectorspol.txt')




In [ ]:
#| export
class Classifier:

    def load_embeddings(file_path):
        '''Loads the embeddings from a file and returns a dictionary with the words as keys and the vectors as values'''
        word_to_vec = {}
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                values = line.split()
                word = values[0]
                vector = np.array(values[1:], dtype='float32')
                print(vector.shape)
                word_to_vec[word] = vector
        return word_to_vec
